In [38]:
import csv
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.keras import Model, layers

In [2]:
def getData():
    with open('process/process_file_3.csv') as csv_file:
        reader = csv.reader(csv_file)
        data = list(reader)

    return data


def preprocessData(data):
    one_hot_encoder = preprocessing.OneHotEncoder()

    processed_data = one_hot_encoder.fit_transform(data[:, 0:2]).toarray()
    processed_data = np.append(processed_data, data[:, 2:6], 1)

    processed_data = preprocessing.normalize(processed_data)
    np.random.shuffle(processed_data)

    return processed_data

In [ ]:
def learn(data):
    data = preprocessData(data)
    num_params = data.shape[1] - 1

    X = data[:, 0:num_params]
    Y = data[:, num_params].reshape(-1, 1)

    # Split the data into training and testing sets (70/30)
    train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.30)
    train_opening_price = train_X[:, num_params - 1].reshape(-1, 1)
    test_opening_price = test_X[:, num_params - 1].reshape(-1, 1)
    
    # Get the initial stock prices for computing the relative cost
    stock_data = tf.placeholder(tf.float32, [None, num_params])
    opening_price = tf.placeholder(tf.float32, [None, 1])
    stock_price = tf.placeholder(tf.float32, [None, 1])

    # Number of neurons in the hidden layer
    n_hidden_1 = 3
    n_hidden_2 = 3

    weights = {
        'out': tf.Variable(tf.random_normal([n_hidden_2, 1]))
    }
    biases = {
        'out': tf.Variable(tf.random_normal([1]))
    }

    # Implement dropout to reduce overfitting
    keep_prob_input = tf.placeholder(tf.float32)
    keep_prob_hidden = tf.placeholder(tf.float32)
    
    #layers
    model = tf.keras.models.Sequential()
    model.add(layers.Conv1D(filters = 32, kernel_size = 3, acitvation = 'relu', input_shape = ()))
    model.add(layers.MaxPool1D(2))
    model.add(layers.Conv1D(filters = 64, kernel_size = 3, activation = 'relu'))
    model.add(layers.MaxPool1D(2))
    model.add(layers.Flatten())
    model.add(layers.Dense(3, activation = 'relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(3, activation = 'relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1))
    
    output_layer = tf.add(tf.matmul(model.output_layer, weights['out']), biases['out'])
    cost_function = tf.reduce_mean(tf.pow(tf.div(tf.subtract(stock_price, output_layer), opening_price), 2))
    optimizers = tf.train.AdamOptimizer(learning_rate).minimize(cost_function)
    
    model.compile(optimizer = optimizers, )

In [48]:
model = tf.keras.models.Sequential()
model.add(layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu', input_shape = (14900, 757, 1)))
model.add(layers.MaxPool2D(2))
model.add(layers.Conv2D(filters = 64, kernel_size = 3, activation = 'relu'))
model.add(layers.MaxPool2D(2))
model.add(layers.Flatten())
model.add(layers.Dense(3, activation = 'relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation = 'relu'))
model.add(layers.Dropout(0.3))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 14898, 755, 32)    320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 7449, 377, 32)     0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7447, 375, 64)     18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3723, 187, 64)     0         
_________________________________________________________________
flatten (Flatten)            (None, 44556864)          0         
_________________________________________________________________
dense (Dense)                (None, 3)                 133670595 
_________________________________________________________________
dropout (Dropout)            (None, 3)                

In [49]:
def learn(data):
    data = preprocessData(data)
    num_params = data.shape[1] - 1

    X = data[:, 0:num_params]
    Y = data[:, num_params].reshape(-1, 1)

    # Split the data into training and testing sets (70/30)
    train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.30)
    train_opening_price = train_X[:, num_params - 1].reshape(-1, 1)
    test_opening_price = test_X[:, num_params - 1].reshape(-1, 1)
    
    # Get the initial stock prices for computing the relative cost
    stock_data = tf.placeholder(tf.float32, [None, num_params])
    opening_price = tf.placeholder(tf.float32, [None, 1])
    stock_price = tf.placeholder(tf.float32, [None, 1])

    # Number of neurons in the hidden layer
    n_hidden_1 = 3
    n_hidden_2 = 3

    weights = {
        'out': tf.Variable(tf.random_normal([n_hidden_2, 1]))
    }
    biases = {
        'out': tf.Variable(tf.random_normal([1]))
    }

    # Implement dropout to reduce overfitting
    keep_prob_input = tf.placeholder(tf.float32)
    keep_prob_hidden = tf.placeholder(tf.float32)

    # Hidden layers
    input_dropout = tf.nn.dropout(stock_data, keep_prob_input)
    conv_l1 = slim.conv1d(input_dropout, 32, 3, activation_fn = tf.nn.relu)
    maxpool_l1 = tf.nn.max_pool1d(conv_l1, 2)
    conv_l2 = slim.conv1d(maxpool_l1, 64, 3, activation_fn = tf.nn.relu)
    maxpool_l2 = tf.nn.max_pool1d(conv_l2, 2)
    flatten = tf.nn.flatten(maxpool_l2)
    layer_1 = slim.fully_connected(flatten, n_hidden_1, biases_initializer=None, activation_fn=tf.nn.relu)
    layer_1_dropout = tf.nn.dropout(layer_1, keep_prob_hidden)
    layer_2 = slim.fully_connected(input_dropout, n_hidden_1, biases_initializer=None, activation_fn=tf.nn.relu)
    layer_2_dropout = tf.nn.dropout(layer_2, keep_prob_hidden)
    
    # regression  layer = (w'x+b)
    output_layer = tf.add(tf.matmul(layer_2_dropout, weights['out']), biases['out'])

    learning_rate = 1e-4
    cost_function = tf.reduce_mean(tf.pow(tf.div(tf.subtract(stock_price, output_layer), opening_price), 2))
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost_function)

    last_cost = 0
    tolerance = 1e-6
    epochs = 1
    max_epochs = 15000

    config = tf.ConfigProto(log_device_placement = True)
    sess = tf.Session(config=config)
    with sess:
        init = tf.global_variables_initializer()
        sess.run(init)

        history = []
        while True:
            sess.run(optimizer,
                     feed_dict={stock_data: train_X.reshape(train_X.shape+(1,)), opening_price: train_opening_price, stock_price: train_Y,
                                keep_prob_input: 0.8, keep_prob_hidden: 0.5})

            if epochs % 100 == 0:
                cost = sess.run(cost_function, feed_dict={stock_data: train_X, opening_price: train_opening_price,
                                                          stock_price: train_Y, keep_prob_input: 0.8,
                                                          keep_prob_hidden: 0.5})
                history.append(cost)
                print("Epoch: %d: Error: %f" % (epochs, cost))

                if abs(cost - last_cost) <= tolerance or epochs > max_epochs:
                    print("Converged.")
                    break
                last_cost = cost

            epochs += 1

        print("Test error: ", sess.run(cost_function, feed_dict={stock_data: test_X.reshape(test_X.shape+(1,)), opening_price: test_opening_price,
                                                                 stock_price: test_Y, keep_prob_input: 1.0,
                                                                 keep_prob_hidden: 1.0}))
        test_results = sess.run(output_layer, feed_dict={stock_data: test_X, stock_price: test_Y, keep_prob_input: 1.0,
                                                         keep_prob_hidden: 1.0})

    avg_perc_error = 0
    max_perc_error = 0
    mei = 0
    for i in range(len(test_Y)):
        actual_change = abs(test_Y[i][0] - test_X[i][num_params - 1]) / test_X[i][num_params - 1]
        predicted_change = abs(test_results[i][0] - test_X[i][num_params - 1]) / test_X[i][num_params - 1]
        delta = abs(actual_change - predicted_change)
        avg_perc_error = avg_perc_error + delta
        if delta > max_perc_error:
            max_perc_error = delta
            mei = i

    avg_perc_error = (avg_perc_error * 100) / len(test_Y)
    max_perc_error *= 100
    print("Maximum percentage error: %f\nAverage percentage error: %f\n" % (max_perc_error, avg_perc_error))


def main():
    data = np.array(getData())
    learn(data)




In [50]:
main()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


ValueError: Convolution expects input with rank 3, got 2

In [16]:
#predicting closing price
#cost fn on opening price 

data = np.array(getData())
data = preprocessData(data)
num_params = data.shape[1] - 1

X = data[:, 0:num_params]
Y = data[:, num_params].reshape(-1, 1)

# Split the data into training and testing sets (70/30)
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.30)
train_opening_price = train_X[:, num_params - 1].reshape(-1, 1)
test_opening_price = test_X[:, num_params - 1].reshape(-1, 1)

In [22]:
data[:,num_params].reshape(-1,1).shape

(21286, 1)

In [23]:
np.linalg.matrix_rank(train_X)

757

In [32]:
train_X.reshape(train_X.shape+(1,)).shape

(14900, 759, 1)

In [39]:
train_X.shape

(14900, 759)

In [45]:
len(train_X)

14900